## Diagnosis

While the purpose of Flaredown for it's users isn't about diagnosis, we would be remiss if we ignored the fact that we have a very solid training set for a system to perform diagnosis.  We will reshape the data so that our features are one-hotted symptoms, and we will predict on condition.

There are a lot of algorithms out there that take a list of self reported symptoms and attempt a diagnosis.  But the depth and breadth of the ever-growing Flaredown data may provide new oppertunities for this task, especially since in the future Flaredown may collect any number of additional variables.

I do recommend giving this some time before it's used.  At this time (Aug 22, 2016) the data only describes about 900 conditions.  For a diagnosis engine to be useful it is likely to require a huge breadth of conditions.

In [ ]:
import numpy as np
import pandas as pd
import random
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import label_ranking_average_precision_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

df = pd.read_csv("flaredown_trackable_data_080316.csv")
df['checkin_date'] = pd.to_datetime(df['checkin_date'])

def combineConditions(x):
    return set(x)

def makeList(x):
    return list(x)

def numericOr(x):
    if 1 in x.values:
        return 1
    else:
        return 0
    
def reshapeSymptoms(df):
    #reshape and one-hot the symptoms

    symptoms = pd.get_dummies(df[(df['trackable_type'] == "Symptom") & (df['trackable_value'] != 0)], columns=['trackable_name'])
    symptoms = symptoms.drop(['trackable_id', 'trackable_type', 'trackable_value'], axis=1)
    symptoms = symptoms.groupby(['user_id', 'checkin_date']).agg(numericOr).reset_index()
    return symptoms
    
def createXY(df, symptoms):
    newdf = df[df['trackable_type'] == 'Condition'].groupby(['user_id', 'checkin_date'])['trackable_name'].agg(combineConditions).reset_index()
    newdf = newdf.merge(symptoms, on=['user_id','checkin_date'])

    #newdf = newdf.drop_duplicates().drop(['user_id','checkin_date','trackable_id','trackable_type', 'trackable_value'], axis=1)
    newdf = newdf.drop(['user_id','checkin_date'], axis=1)
    X = newdf.drop('trackable_name', axis=1)
    Y = newdf['trackable_name'].apply(makeList)  # each row of Y is a list, because this is a multilabel problem
    mlb = MultiLabelBinarizer()
    Y = mlb.fit_transform(Y)  
    return train_test_split(X, Y, test_size=0.2, random_state=42)  

In [109]:
symptoms = reshapeSymptoms(df)
X_train, X_test, Y_train, Y_test = createXY(df, symptoms)

### A note about model selection

It is important to note that in this case Y is a matrix.  This is an example of multi-label classification, in that each user may be suffering from any number of conditions.  Because of this, a classifier that can handle multi-label must be used.  Fortunately, sklearn has several options for this including:

Decision Trees, Random Forests, Nearest Neighbors, Ridge Regression

TODO should try all of the above - but the one-hotted data is waaay sparse, are any of those really going to cut it?  Maybe need to think about PCA here before trying to classify

### Resources

http://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_multilabel_classification.html#sklearn.datasets.make_multilabel_classification

http://scikit-learn.org/stable/modules/multiclass.html


In [70]:
def meanF1(y, pred):
    row_f1 = np.zeros((y.shape[0]))
    for k in range(y.shape[0]):
        tp = (pred[k] * y[k]).sum()
        fp = (pred[k] > y[k]).sum()
        fn = (pred[k] < y[k]).sum()
        precision = tp/(tp + fp + 1e-9)
        recall = tp/(tp + fn + 1e-9)
        row_f1[k] = 2 * precision * recall / (precision + recall + 1e-9)
    return np.mean(row_f1)

def scorePredictions(X_train, X_test, Y_train, Y_test):
    rf = OneVsRestClassifier(RandomForestClassifier(n_estimators=10, criterion='gini', max_depth=3))
    rf.fit(X_train, Y_train)
    Y_pred = rf.predict(X_test)
    print meanF1(Y_test, Y_pred)
    #print label_ranking_average_precision_score(Y_test, Y_pred)
    print mlb.inverse_transform(Y_test)[1]
    print mlb.inverse_transform(Y_pred)[1]
    


In [32]:
scorePredictions(X_train, X_test, Y_train, Y_test)

0.109695248449
('Celiac disease', 'Collagenous colitis', 'Complex partial seizures', 'Fatigue', 'GERD', 'Headaches', 'Interstitial cystitis', 'Lyme disease', 'Osteoarthritis', 'Osteoporosis', 'Scleroderma')
('Celiac disease', 'Complex partial seizures', 'Fatigue', 'GERD', 'Headaches', 'Lyme disease', 'Osteoarthritis', 'Scleroderma')


OK first off, that score is not pretty.  1 is the optimal score for that metric.  At least it will be easy to beat the benchmark...

Also, those warnings are a legit problem.  What's happening is, some conditions only appear in the data once, and thus they can't be represented in both the training and test sets.  I'm going to have to filter out those conditions and not attempt to predict them.  And unfortunately, I'm going to have to not predict on the entire user, since deleting one of their conditions renders the rest of their symptom/condition relationships invalid.  As usual, as the data grows this should become less of a problem.

In [76]:
#Since the original DF is one condition per line, it's easier to filter there and recreate our X and Y
def filterCondition(x):
    keep = True
    for value in x:
        if value in onceoffs:
            keep = False
    return keep

condition_counts = df[df['trackable_type'] == 'Condition']['trackable_name'].value_counts()
onceoffs = list(condition_counts[condition_counts < 2].keys())
#cleandf = df.groupby('user_id').filter(lambda x: x not in onceoffs)
cleandf = df.groupby('user_id').filter(filterCondition)
print "deleting " + str(len(onceoffs)) + " users who have unique condition values"


deleting 214 users who have unique condition values


In [110]:
symptoms = reshapeSymptoms(cleandf)
X_train, X_test, Y_train, Y_test = createXY(cleandf, symptoms)
scorePredictions(X_train, X_test, Y_train, Y_test)

0.109060572185
('Celiac disease', 'Collagenous colitis', 'Complex partial seizures', 'Fatigue', 'GERD', 'Headaches', 'Interstitial cystitis', 'Lyme disease', 'Osteoarthritis', 'Osteoporosis', 'Scleroderma')
('Collagenous colitis', 'Complex partial seizures', 'Fatigue', 'GERD', 'Headaches', 'Interstitial cystitis', 'Lyme disease', 'Osteoarthritis', 'Osteoporosis', 'Scleroderma')


So it's producing some reasonable results, but the score still sucks mightily.

In a similar problem, [this guy](https://github.com/davidthaler/Greek_media) had good luck with running his Y through PCA and using a Ridge regressor.  I'm going to give that a try.

In [111]:
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge

def predict(decision, threshold1, threshold2):
    pred = ((decision - threshold1) > 0).astype(float)
    max_decision = decision.max(1)
    for k in range(pred.shape[0]):
        cut = max_decision[k] - threshold2
        idx = (decision[k, :] >= cut) 
        pred[k, idx] = 1
    return pred

def scoreRidge(pca_comps, thresh1, thresh2, ridge_alpha):
    pcaobj = PCA(n_components=pca_comps)  
    Y_train_transformed = pcaobj.fit_transform(Y_train)
    rig = Ridge(alpha=ridge_alpha) 
    rig.fit(X_train, Y_train_transformed)

    prediction = rig.predict(X_test)
    pred = pcaobj.inverse_transform(prediction)
    Y_pred = predict(pred,thresh1, thresh2)
    return meanF1(Y_test, Y_pred)

def predictRidge(pca_comps, thresh1, thresh2, ridge_alpha):
    pcaobj = PCA(n_components=pca_comps)  
    Y_train_transformed = pcaobj.fit_transform(Y_train)
    rig = Ridge(alpha=ridge_alpha) 
    rig.fit(X_train, Y_train_transformed)

    prediction = rig.predict(X_test)
    pred = pcaobj.inverse_transform(prediction)
    Y_pred = predict(pred,thresh1, thresh2)
    return Y_pred

#all these hypers are arbitrary, will optimize later if it works out
score = scoreRidge(10,0.3,0.1,1)
print score

0.451828111003


That's around 4x better than my benchmark... before even optimizing.  And I only expected around %50 F1 to be possible anyway.

Time to optimize.  I'm going to grid search, because ridge regression runs so fast anyways.

In [95]:
def findMaxScore(pca_comps,thresh1s,thresh2s,ridge_alphas):
    max_score = 0
    for pca_comp in pca_comps:
        for thresh1 in thresh1s:
            for thresh2 in thresh2s:
                for ridge_alpha in ridge_alphas:
                    score = scoreRidge(pca_comp,thresh1,thresh2,ridge_alphas)
                    if score > max_score:
                        max_score = score
                        print "PCA Comps " + str(pca_comp) + " thresh1 " + str(thresh1) + " thresh2 " + str(thresh2) + " ridge_alpha " + str(ridge_alpha) +" score" + str(score)
pca_comps = [5,10,15,20,25,30]
thresh1s = [0.05,0.1,0.5]
thresh2s = [0.05,0.1,0.5]
ridge_alphas = [1]
findMaxScore(pca_comps,thresh1s,thresh2s,ridge_alphas)

PCA Comps 5 thresh1 0.05 thresh2 0.05 ridge_alpha 1 score0.196211012586
PCA Comps 5 thresh1 0.1 thresh2 0.05 ridge_alpha 1 score0.289850873118
PCA Comps 5 thresh1 0.5 thresh2 0.05 ridge_alpha 1 score0.353871577766
PCA Comps 10 thresh1 0.5 thresh2 0.05 ridge_alpha 1 score0.439993263063
PCA Comps 15 thresh1 0.5 thresh2 0.05 ridge_alpha 1 score0.503744515188
PCA Comps 20 thresh1 0.5 thresh2 0.05 ridge_alpha 1 score0.559917903774
PCA Comps 25 thresh1 0.5 thresh2 0.05 ridge_alpha 1 score0.622782453263
PCA Comps 30 thresh1 0.5 thresh2 0.05 ridge_alpha 1 score0.659664689019


In [97]:
#Some of the optimal values found were at the max range, so just going to do that again with some higher/lower values

pca_comps = [35,40,45,50]
thresh1s = [0.5,1,1.5]
thresh2s = [0.01,0.05]
ridge_alphas = [1]
findMaxScore(pca_comps,thresh1s,thresh2s,ridge_alphas)

PCA Comps 35 thresh1 0.5 thresh2 0.01 ridge_alpha 1 score0.681774234408
PCA Comps 35 thresh1 0.5 thresh2 0.05 ridge_alpha 1 score0.684301326599
PCA Comps 40 thresh1 0.5 thresh2 0.01 ridge_alpha 1 score0.710495957352
PCA Comps 45 thresh1 0.5 thresh2 0.01 ridge_alpha 1 score0.728313727167
PCA Comps 50 thresh1 0.5 thresh2 0.01 ridge_alpha 1 score0.746364508545


In [100]:
pca_comps = [100,150,200,300,400]
thresh1s = [0.5,0.6,0.7,0.8]
thresh2s = [0.001,0.01]
ridge_alphas = [1]
findMaxScore(pca_comps,thresh1s,thresh2s,ridge_alphas)

PCA Comps 100 thresh1 0.5 thresh2 0.001 ridge_alpha 1 score0.803389862461
PCA Comps 100 thresh1 0.5 thresh2 0.01 ridge_alpha 1 score0.80477609451
PCA Comps 150 thresh1 0.5 thresh2 0.001 ridge_alpha 1 score0.836694243113
PCA Comps 150 thresh1 0.5 thresh2 0.01 ridge_alpha 1 score0.838125110049
PCA Comps 200 thresh1 0.5 thresh2 0.001 ridge_alpha 1 score0.847978972387
PCA Comps 200 thresh1 0.5 thresh2 0.01 ridge_alpha 1 score0.849965862481
PCA Comps 300 thresh1 0.5 thresh2 0.001 ridge_alpha 1 score0.854075770037
PCA Comps 300 thresh1 0.5 thresh2 0.01 ridge_alpha 1 score0.855022029633
PCA Comps 400 thresh1 0.5 thresh2 0.001 ridge_alpha 1 score0.856714751009
PCA Comps 400 thresh1 0.5 thresh2 0.01 ridge_alpha 1 score0.857671212865


In [102]:
pca_comps = [400,500,600,700,800]
thresh1s = [0.5]
thresh2s = [0.01]
ridge_alphas = [1]
findMaxScore(pca_comps,thresh1s,thresh2s,ridge_alphas)

PCA Comps 400 thresh1 0.5 thresh2 0.01 ridge_alpha 1 score0.857671212865
PCA Comps 500 thresh1 0.5 thresh2 0.01 ridge_alpha 1 score0.857834038136
PCA Comps 600 thresh1 0.5 thresh2 0.01 ridge_alpha 1 score0.858864297086


In [105]:
pca_comps = [500,550,600,625,650]
thresh1s = [0.5]
thresh2s = [0.01]
ridge_alphas = [1]
findMaxScore(pca_comps,thresh1s,thresh2s,ridge_alphas)

PCA Comps 500 thresh1 0.5 thresh2 0.01 ridge_alpha 1 score0.857834038136
PCA Comps 550 thresh1 0.5 thresh2 0.01 ridge_alpha 1 score0.858797344756
PCA Comps 600 thresh1 0.5 thresh2 0.01 ridge_alpha 1 score0.858864297086
PCA Comps 625 thresh1 0.5 thresh2 0.01 ridge_alpha 1 score0.858936589744


At this time there are only 808 classes in Y, and we reduce it to 625.  So PCA isn't reducing that much, but as we can see above it's still providing some great benefits.

The final score of 0.8589 is far beyond what I thought would be possible.

Finally, lets take a look at what we can do with these results.  The question I'd like to answer is, for the misclassified users, how misclassified were they, and was there something unusual about these users?

In [123]:
Y_pred = predictRidge(625,0.5,0.01,1)

In [128]:
longer = []
shorter = []
same = []
Y_test_classes = mlb.inverse_transform(Y_test)
Y_pred_classes = mlb.inverse_transform(Y_pred)
for i in range(len(Y_pred)):
    if not (Y_pred[i] == Y_test[i]).all():
        if len(Y_pred_classes[i]) > len(Y_test_classes[i]):
            longer.append(Y_pred_classes[i])
        elif len(Y_pred_classes[i]) < len(Y_test_classes[i]):
            shorter.append(Y_pred_classes[i])
        else:
            same.append(Y_pred_classes[i])
print "users where we guessed they had a condition that they aren't reporting " + str(len(longer))
print "users where we guessed that they didn't have a condition that they are reporting " + str(len(shorter))
print "users where we guessed different conditions than they are reporting " + str(len(same))

users where we guessed they had a condition that they aren't reporting 281
users where we guessed that they didn't have a condition that they are reporting 642
users where we guessed different conditions than they are reporting 144


In [146]:

misclass_pred = []
misclass_test = []
misclass_index = []
for i in range(len(Y_pred)):
    if not (Y_pred[i] == Y_test[i]).all():
        if len(Y_pred_classes[i]) == len(Y_test_classes[i]):
            misclass_pred.append(Y_pred_classes[i])
            misclass_test.append(Y_test_classes[i])
            misclass_index.append(i)
print misclass_pred[0]
print misclass_test[0]
            
print misclass_pred[1]
print misclass_test[1]

print misclass_pred[2]
print misclass_test[2]

('Asthma',)
('Acne',)
('Chronic fatigue syndrome',)
("Crohn's disease",)
('Chronic fatigue syndrome',)
("Hashimoto's disease",)


Just looking at these first few, it seems like the errors it's making are highly unrelated conditions.  Lets take a look at the symptoms these users reported.

In [170]:
misclass_X = X_test.iloc[misclass_index,:]

def printSymptoms(index):
    user = misclass_X.iloc[index,:]
    print "user reports " + str(misclass_test[index])
    print "we predict " + str(misclass_pred[index])
    print "their symptoms are " 
    print user[user.values==1]
    print "\n"

printSymptoms(2)
printSymptoms(10)
printSymptoms(50)
printSymptoms(100)

user reports ("Hashimoto's disease",)
we predict ('Chronic fatigue syndrome',)
their symptoms are 
trackable_name_Dull            1.0
trackable_name_Joint pain      1.0
trackable_name_Muscle pain     1.0
trackable_name_itching face    1.0
Name: 4547, dtype: float64


user reports ('Ulcerative colitis',)
we predict ('Fatigue',)
their symptoms are 
trackable_name_Crying                           1.0
trackable_name_Post-op shoulder surgery pain    1.0
trackable_name_Shooting pain in legs            1.0
trackable_name_Stomach Pain                     1.0
Name: 6137, dtype: float64


user reports ("Crohn's disease",)
we predict ('Idiopathic hypersomnia',)
their symptoms are 
trackable_name_Abdominal pain    1.0
trackable_name_Anxiety           1.0
trackable_name_Self Harm         1.0
trackable_name_sleepiness        1.0
Name: 5034, dtype: float64


user reports ('Arthritis', 'Endometriosis', 'Fibromyalgia')
we predict ('Arthritis', 'Chronic pelvic pain', 'Vulvodynia')
their symptoms are 
tr

I'm not sure how to quantify this result, if anybody has any ideas let me know.  But just looking at these examples by hand, it seems like our predictions tend to make more sense than what they users are reporting!  This gives me high-hopes that even with the current small dataset, we could be making really good recommendations about conditions that a user might want to look into.

It's difficult to interpret the results.  Sometimes it seems like the predicted condition should replace the existing condition, but sometimes it seems like it should be added on.
The first in this list really stands out for me, as a person suffering from Hashimoto's shouldn't even have symptoms if they're on their meds.  That seems to lend credability to our prediction, as it really seems likely that this user has more going on with their health than just Hashimoto's.
The example where we predict 'Idiopathic hypersomnia' is interesting because that user is reporting very little in the way of Crohn's symptoms.  But presumably a person that reports Crohn's has been diagnosed with this by a doctor.  So in general I think it's best to err on the side of caution and not ever suggest that a user's existing condtions are mis-diagnosis.

One more point of interest is that these predictions sometimes say interesting things about the conditions that they are predicting.  In the case of the user above that we predicted Idiopathic hypersomnia for, our diagnosis seems to be based on the report of 'self harm'.  Self harm doesn't seem like a symptom that one would normal associate with idiopathic hypersomnia.  And yet, when we look at the data, this symptom is in fact very commonly paired with this condition.